# Checking Ranges of IRL Alpha Using GPR Trained on Simulation
We had 2 hypothesis on why GPR was having a bad prediction confidence on IRL TFO PPG data.  
1. We do not have enough TMPs or knobs with which we can correctly guess the model
2. The range is not sufficient for the TMPs currently is use, leading to extrapolation rather than interpolation

In this notebook, we try to solve problem#2. The way we do this is by training a GPR on current simulation data and predict on a set of real data. The prediction gives clues as to which range we need to explore. Then based on that, we update our simulation space and re-iterate. Hopefully, at somepoint we should be able to reach interpolation.

## GPR Prediction Method
Theoretically, the TMPs should be independent. (Although I have some doubts based on how independece is actually measured in GPR - via a custom covariance metric using the spatial intensity fitting params. Which in my opinion, should show dependence if  multiple combinations can produce the same distance metric). Based on this independent nature, it is safe to predict each TMP with its own individual GPR model. Keep in mind, everything needs to be zero-mean, unit variance. In this notebook, I train my Scaler on the training data and use that directly on TFO PPG. But if we ever want to include TFO PPG into the training itself, this method needs to be changed.

## Results
It does seem that we are capturing the correct range since the predictions are still within the values. But somehow always predicts the same thing. This might not be such a good test

In [40]:
from TFO_dataset import SheepData
from math import pi
from sklearn.gaussian_process import *
from inverse_modelling_tfo.data.intensity_interpolation import interpolate_exp_chunk, get_interpolate_fit_params
from inverse_modelling_tfo.data import normalize_zero_mean 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from typing import Union, List

In [41]:
# train_data = pd.read_pickle(r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/intensity_averaged_sim_data.pkl')
train_data = pd.read_pickle(
    r'/home/rraiyan/personal_projects/tfo_inverse_modelling/data/intensity/intensity_summed_sim_data_equidistance_detector2.pkl')
print(len(train_data))
# Only for intensity_summed_sim_data_equidistance_detector.pkl
sdd = train_data['SDD'].to_numpy()[:20]
detector_count = [11, 16, 22, 27, 32, 38, 43, 48, 53,
                  59, 64, 69, 75, 80, 85, 90, 96, 101, 106, 111]
sdd_to_detector_count_map = {
    dist: count for dist, count in zip(sdd, detector_count)}
train_data['Intensity'] /= train_data['SDD'].map(
    sdd_to_detector_count_map)

# For the other cases
# train_data['Intensity'] /= 20   # Normalize by the number of detectors per ring

train_data['Intensity'] /= 1e9  # Photon count/Initial intensity



interpolated_training_data = get_interpolate_fit_params(
    train_data, weights=[1, -1])

interpolated_training_data.head()

,Wave Int,Uterus Thickness,Maternal Wall Thickness,Maternal Mu_a,Fetal Mu_a,alpha0,alpha1,alpha2,alpha3
0,2.0,5.0,26.0,0.005,0.05,98.489193,-2.926028,93.351279,-171.975994
1,2.0,5.0,26.0,0.006,0.05,102.376438,-3.064418,97.312697,-178.975132
2,2.0,5.0,26.0,0.007,0.05,106.081459,-3.196045,101.087085,-185.645325
3,2.0,5.0,26.0,0.008,0.05,109.625911,-3.321710,104.696132,-192.024911
4,2.0,5.0,26.0,0.009,0.05,113.027654,-3.442082,108.157935,-198.145779


In [42]:

# Incorporate both wavelengths by moving to a Wide Format from Long Format
interpolated_training_data = interpolated_training_data.pivot_table(
    index=['Uterus Thickness', 'Maternal Wall Thickness', 'Maternal Mu_a', 'Fetal Mu_a'], columns='Wave Int', values=['alpha0', 'alpha1', 'alpha2', 'alpha3']).reset_index()


print(interpolated_training_data.columns)

def _renaming_func(x, y):
    if y == '':
        return f'{x}'
    else:
        return f'{x}_{int(y)}'


interpolated_training_data.columns = [_renaming_func(
    x, y) for x, y in interpolated_training_data.columns]
interpolated_training_data.head()


MultiIndex([(       'Uterus Thickness',  ''),
            ('Maternal Wall Thickness',  ''),
            (          'Maternal Mu_a',  ''),
            (             'Fetal Mu_a',  ''),
            (                 'alpha0', 1.0),
            (                 'alpha0', 2.0),
            (                 'alpha1', 1.0),
            (                 'alpha1', 2.0),
            (                 'alpha2', 1.0),
            (                 'alpha2', 2.0),
            (                 'alpha3', 1.0),
            (                 'alpha3', 2.0)],
           names=[None, 'Wave Int'])


,Uterus Thickness,Maternal Wall Thickness,Maternal Mu_a,Fetal Mu_a,alpha0_1,alpha0_2,alpha1_1,alpha1_2,alpha2_1,alpha2_2,alpha3_1,alpha3_2
0,5.0,2.0,0.005,0.05,113.221144,131.476209,-3.409377,-4.078752,106.769431,122.767976,-196.437138,-225.582066
1,5.0,2.0,0.005,0.06,114.708633,136.732862,-3.485477,-4.290443,108.613750,128.726450,-199.483973,-235.785531
2,5.0,2.0,0.005,0.07,116.106105,141.064586,-3.554691,-4.464508,110.322606,133.631160,-202.321745,-244.187574
3,5.0,2.0,0.005,0.08,117.423466,144.738851,-3.618310,-4.611828,111.916476,137.786931,-204.979218,-251.309266
4,5.0,2.0,0.005,0.09,118.667929,147.922095,-3.677211,-4.739188,113.409611,141.383590,-207.476654,-257.475002


In [51]:
# Non-Normalized Training data
interpolated_training_data.describe()

,Uterus Thickness,Maternal Wall Thickness,Maternal Mu_a,Fetal Mu_a,alpha0_1,alpha0_2,alpha1_1,alpha1_2,alpha2_1,alpha2_2,alpha3_1,alpha3_2,Bias Ratio
count,475.0,475.000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000,475.000000
mean,5.0,20.000,0.007000,0.070000,113.757879,118.257236,-3.470978,-3.699648,109.017676,114.461372,-199.609237,-208.632460,0.975197
std,0.0,10.966,0.001416,0.014157,7.197941,18.694420,0.284560,0.764378,7.725566,20.343442,13.384610,35.040578,0.087413
min,5.0,2.000,0.005000,0.050000,101.210185,97.028772,-4.149646,-5.853067,95.806122,91.751952,-231.712818,-308.248897,0.732893
25%,5.0,10.000,0.006000,0.060000,108.486387,105.693772,-3.634716,-4.052920,104.102567,100.630894,-207.259792,-224.509410,0.930474
50%,5.0,20.000,0.007000,0.070000,113.837463,112.565299,-3.447280,-3.434677,108.769624,107.658676,-199.420592,-197.276481,1.027122
75%,5.0,30.000,0.008000,0.080000,118.130341,126.483307,-3.292540,-3.179258,113.281405,123.668910,-190.694314,-184.873533,1.035254
max,5.0,38.000,0.009000,0.090000,131.180255,171.336254,-2.996616,-2.871157,127.471357,172.229066,-176.537592,-169.205054,1.046877


In [43]:
# Fitting only on both WV

# Create Features & Normalize the fitting params
# interpolated_training_data['Bias Ratio'] = interpolated_training_data['alpha0_1'] / interpolated_training_data['alpha0_2']

# X = interpolated_training_data[['Bias Ratio', 'alpha1_1', 'alpha1_2', 'alpha2_1', 'alpha2_2', 'alpha3_1', 'alpha3_2']].to_numpy()
X = interpolated_training_data[['alpha0_1', 'alpha0_2', 'alpha1_1', 'alpha1_2', 'alpha2_1', 'alpha2_2', 'alpha3_1', 'alpha3_2']].to_numpy()
alpha_scaler = preprocessing.StandardScaler().fit(X)
X = alpha_scaler.transform(X)

y = interpolated_training_data[['Fetal Mu_a']].to_numpy().flatten()
# y = interpolated_training_data[['Maternal Mu_a']].to_numpy().flatten()
y_scaler = preprocessing.StandardScaler().fit(y.reshape(-1, 1))
y = y_scaler.transform(y.reshape(-1, 1)).flatten()

rng = np.random.RandomState(1)
random_indices = rng.choice(np.arange(y.size), size=y.size, replace=False)
training_count = int(y.size * 1)  # 80% Training Data
training_indices = random_indices[:training_count]
test_indices = random_indices[training_count:]

X_train, y_train = X[training_indices], y[training_indices]
X_test, y_test = X[test_indices], y[test_indices]

In [44]:
# kernel = 1 * kernels.RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e1))
kernel = 1 * kernels.Matern(length_scale=1.0, length_scale_bounds=(1e-6, 1e-2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gp.fit(X_train, y_train)
gp.kernel_

1.03**2 * Matern(length_scale=5.04e-05, nu=1.5)

In [45]:
# # PRedict on Simulation
# X_test = X_train
# y_test = y_train
# mean_prediction, std_prediction = gp.predict(X_test, return_std=True)
# mae = np.abs(mean_prediction - y_test)
# mse = np.square(mean_prediction - y_test)
# df = pd.DataFrame(
#     {
#         'True a0' : X_test[:, 0],
#         'True a1' : X_test[:, 1],
#         'True a2' : X_test[:, 2],
#         'True a3' : X_test[:, 3],
#         'True y'  : y_test,
#         'Prediction' : mean_prediction,
#         'Confidence' : std_prediction,
#         'MAE(%)' : mae * 100,
#         'MSE(%)' : mse * 100,
#     }
# )
# pd.set_option('display.max_rows', 1200)
# df

In [46]:
# df['MAE(%)'].describe()

In [47]:
def prepare_patient_ppg(ppg_data : pd.DataFrame, sample_number : Union[int, List], SDD = [15, 30, 45, 70, 100]) -> np.ndarray:
    """Prepare PPG data to be used directly into the GPR prediction.

    Args:
        ppg_data (pd.DataFrame): PPG data Dataframe. You can feed data directly from the the TFO_dataset package.
        (Note: This should ideally be the optically normalized data)
        sample_number (int): which sample to choose. You can either pass a single integer or an array
        SDD (_type_, optional): Detector distances in TFO device(in mm). Defaults to SDD=[15, 30, 45, 70, 100].
    """
    # The code is generalized to run on any array. make necessary conversions 
    if isinstance(sample_number, int):
        sample_number = [sample_number]
    
    patient_features = []
    for sample_point in sample_number:
        # Pick a point in time
        spatial_intensity = ppg_data.iloc[sample_point].copy() 
        spatial_intensity *=  pi * 4   # from unit area -> pi r^2 area -> match simulation
        # Reshape ppg data to fit the format
        spatial_intensity_wv1 = pd.DataFrame(data={
            'SDD' : SDD,
            'Intensity' : spatial_intensity.to_numpy()[:5]
        })
        spatial_intensity_wv2 = pd.DataFrame(data={
            'SDD' : SDD,
            'Intensity' : spatial_intensity.to_numpy()[5:]
        })
        alpha_wv1 = interpolate_exp_chunk(spatial_intensity_wv1, weights=[1.0, -1.0], return_alpha=True).flatten()
        alpha_wv2 = interpolate_exp_chunk(spatial_intensity_wv2, weights=[1.0, -1.0], return_alpha=True).flatten()
        patient_features.append([alpha_wv1[0], alpha_wv2[0], alpha_wv1[1], alpha_wv2[1], alpha_wv1[2], alpha_wv2[2], alpha_wv1[3], alpha_wv2[3]])
        
    return np.array(patient_features)
    

## Variable Part of the Data
Everything above only needs to be run once!

In [56]:
# Predict on reallife data
tag = {'experiment_number': 11, 'experiment_round': 1, 'experiment_year_prefix': 'sp2022',
       'additional_info': '', 'data_version': 'iq_demod_optical'}
data = SheepData('iq_demod_optical').get_data_from_tag(tag)
print(f'Sample Length : {len(data)}')

# Pick 20 equidistance points within the length
point_count = 20
sample_numbers = np.linspace(100, len(data) - 1, point_count)
sample_numbers = [int(x) for x in sample_numbers]
features = prepare_patient_ppg(data, sample_numbers)

# Create a DF for better viz.
print('Non- normalized Features')
feature_names = [f'f{i + 1}' for i in range(8)]
features = alpha_scaler.transform(features)
print(pd.DataFrame(features, columns=feature_names))

Sample Length : 217120
Non- normalized Features
           f1        f2         f3        f4         f5        f6         f7  \
0  -22.592250 -8.590743  14.447283  5.615544 -18.680464 -7.176588  20.051632   
1  -21.557080 -8.056010  13.981027  5.385607 -17.941689 -6.801068  19.215122   
2  -22.336084 -8.488919  14.343910  5.579962 -18.506633 -7.110346  19.850355   
3  -21.960485 -8.322790  14.172400  5.507752 -18.237542 -6.993676  19.546692   
4  -20.744321 -7.514747  13.627232  5.164015 -17.371986 -6.430865  18.567612   
5  -22.052601 -8.363931  14.207150  5.519520 -18.299443 -7.018544  19.618696   
6  -21.781283 -8.234477  14.092928  5.469957 -18.109868 -6.931818  19.402170   
7  -22.582861 -8.763239  14.448131  5.697629 -18.676211 -7.302305  20.044599   
8  -22.247459 -8.552147  14.298128  5.606241 -18.438763 -7.154437  19.775424   
9  -23.432404 -9.114990  14.843910  5.856150 -19.292872 -7.554085  20.738030   
10 -22.136948 -8.353189  14.253825  5.529985 -18.363609 -7.021503  19.68

In [57]:
estimate, confidence = gp.predict(features, return_cov=True)
# estimate, confidence = gp.predict(X_train[0, :].reshape(1, -1), return_std=True)
print(y_scaler.inverse_transform(np.array(estimate).reshape(-1, 1)))
print(pd.DataFrame(confidence))

[[0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]
 [0.07]]
          0         1         2         3         4         5         6   \
0   1.061228  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1   0.000000  1.061228  0.000000  0.000000  0.000000  0.000000  0.000000   
2   0.000000  0.000000  1.061228  0.000000  0.000000  0.000000  0.000000   
3   0.000000  0.000000  0.000000  1.061228  0.000000  0.000000  0.000000   
4   0.000000  0.000000  0.000000  0.000000  1.061228  0.000000  0.000000   
5   0.000000  0.000000  0.000000  0.000000  0.000000  1.061228  0.000000   
6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  1.061228   
7   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
8   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
9   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
10 

In [50]:
print(f'alpha means (From training) : {alpha_scaler.mean_}')
print(f'alpha variance (From training) : {alpha_scaler.var_}')

alpha means (From training) : [ 113.75787911  118.25723638   -3.47097802   -3.69964811  109.01767627
  114.46137244 -199.60923674 -208.6324601 ]
alpha variance (From training) : [5.17012756e+01 3.48745582e+02 8.08040191e-02 5.83043774e-01
 5.95587113e+01 4.12984369e+02 1.78770619e+02 1.22525721e+03]
